# Plan‐Project Classification Validation

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import json
from openai import OpenAI

In [ ]:
BASE_PATH = "../data/"

In [ ]:
description_df = pd.read_csv(BASE_PATH + "raw/1-2_RS_2024_基本情報_事業概要等.zip")
sme_description_df = description_df.query(
    "主要経費 == '中小企業対策費' and 政策所管府省庁 == '経済産業省'"
).reset_index(drop=True)
kpi_df = pd.read_csv(BASE_PATH + "raw/3-1_RS_2024_効果発現経路_目標・実績.zip").query(
    "予算事業ID in @id_list"
)
check_df = pd.read_csv(BASE_PATH + "raw/4-1_RS_2024_点検・評価.zip").query(
    "予算事業ID in @id_list"
)
amount_df = pd.read_csv(BASE_PATH + "raw/2-1_RS_2024_予算・執行_サマリ.zip").query(
    "予算事業ID in @id_list and 会計区分 == '一般会計' and 予算年度 == 2024"
)[["事業年度", "予算事業ID", "当初予算"]]

In [ ]:
dtm = sme_description_df[
    [
        "事業年度",
        "予算事業ID",
        "事業名",
        "政策所管府省庁",
        "事業の目的",
        "現状・課題",
        "事業の概要",
        "事業区分",
        "事業開始年度",
        "事業終了（予定）年度",
    ]
]
dtm = dtm.merge(amount_df, on=["事業年度", "予算事業ID"]).query("当初予算 > 0")

## LLM  classification

In [ ]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=userdata.get("OPENAI_API_KEY"),
)

### 予算事業に関する分類質問

In [ ]:
str_output = dtm[["予算事業ID", "output"]].to_string(index=False)
content_text = """
あなたはとても日本の国家公務員であり、各省庁の予算事業の分類を行います.\n
入力したデータを読み取って、各予算事業IDに事業分類を割り振ってください\n
与えられたデータフレームは 予算事業ID: 予算事業の識別ID、事業の概要: 「当該事業の内容」　です。\n

事業分類(class)は、以下の3つから選んでください。\n
基本的にPDCAで考えてください。\n

# 事業分類の考え方
## 軸１：PDCAにおけるDoなのか? Planなのか？
[Do]
- 直接社会への介入を行っているもの
- 補助金や広報など、社会に対してなんらかの行動変容を促すことを目指す事業
[Plan]
- 介入そのもの調査　（PoCや調査事業）
- 介入そのものコンセプト（概念）の実証（本当にこの介入って必要なのか？現実的なのか？）
- 介入の事前調査など
[その他]
- 経費、人件費　等


# 事業分類
1. Do事業\n
目的：
直接社会への介入を行っているもの
社会（市民、企業、他官庁等）の行動や意識に変化をもたらすことを目指す事業\n
例： 広報・啓発キャンペーン、インセンティブ提供型プログラム、規制や制度改正の導入支援など\n

2. Plan事業\n
- Do事業の事前事業に該当するような事業
- エビデンスギャップを埋めるための実験もこれに含める
- 介入そのもの調査　（PoCや調査事業）
- 介入そのものコンセプト（概念）の実証（本当にこの介入って必要なのか？現実的なのか？）
- 介入の事前調査など
- 「実証事業」や「調査事業」といった文言が入る可能性が高い
- また社会に直接働きかけをしていても、当該施策の対象者がごく少数であれば、それは実証事業として「Plan事業」と見なしなさい

3. その他\n
目的： 上記３類型に該当しない、日常的な行政運営や補助業務、内部管理・支援業務など\n
例： 業務プロセスの改善、内部研修、庶務的な業務、事務管理など\n

# 出力はdict ※これ以外は絶対出力しないで！(不足説明は絶対不要)\n
- ID: もとのデータフレームの予算事業IDに対応\n
- class: あなたが付与した分類名（日本語で）\n
# 出力形式 ※これ以外は絶対出力しないで！\n
[
    {"ID" : 3503, "class": "Plan事業"},
    {"ID" : 3504, "class": "Do事業"},
    {"ID" : 3505, "class": "その他"},
]
"""


completion = client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {"role": "developer", "content": content_text},
        {"role": "user", "content": f"以下のデータフレームが経済産業省のデータになります\n{str_output}"},
    ],
)


chatgpt_class_dict = json.loads(
    completion.choices[0].message.content.strip("```json\n").strip("```")
)
chatgpt_class = pd.DataFrame(chatgpt_class_dict)
chatgpt_class.columns = ["予算事業ID", "llm_class"]
dtm_with_class = dtm.merge(chatgpt_class, on="予算事業ID", how="left")

In [ ]:
# サンプルチェック
dtm_with_class.query("予算事業ID == 6080")

In [ ]:
dtm_with_class.groupby("llm_class")["予算事業ID"].count()

In [ ]:
len(dtm_with_class) # 50

## vs.  our simplified classification

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import pprint

In [ ]:
plan_words_list = ["実証", "PoC", "poc", "POC", "POC", "概念実証", "調査", "調査事業"]
# plan_words_listを'|'.join()で結合
pattern = "|".join(plan_words_list)

# 'output'列に対してcontainsを適用し、結果を0または1に変換
dtm_with_class["plan_project"] = (
    dtm_with_class["output"].str.contains(pattern, case=False, na=False).astype(int)
)
dtm_with_class["llm_plan_project"] = np.where(
    dtm_with_class["llm_class"] == "Plan事業", 1, 0
)

In [ ]:
print(
    classification_report(
        dtm_with_class["llm_plan_project"], dtm_with_class["plan_project"]
    )
)

In [ ]:
print(
    confusion_matrix(dtm_with_class["llm_plan_project"], dtm_with_class["plan_project"])
)